In [150]:
import os
import json
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
import re


In [151]:
'''test_notepad_path = 'phase1/annotations/humans_and_advanced_bots/test.txt'
train_notepad_path = 'phase1/annotations/humans_and_advanced_bots/test.txt'
json_directory_path = 'phase1/data/mouse_movements/humans_and_advanced_bots'
session_folders = os.listdir(json_directory_path)'''

"test_notepad_path = 'phase1/annotations/humans_and_advanced_bots/test.txt'\ntrain_notepad_path = 'phase1/annotations/humans_and_advanced_bots/test.txt'\njson_directory_path = 'phase1/data/mouse_movements/humans_and_advanced_bots'\nsession_folders = os.listdir(json_directory_path)"

In [152]:

'''# Function to read session IDs and their labels
def read_session_labels(file_path):
    session_labels = {}
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            if len(parts) == 2:
                session_id, label = parts
                session_labels[session_id] = label
    return session_labels

# Read session IDs and labels from both test and train files
test_session_labels = read_session_labels(test_notepad_path)
train_session_labels = read_session_labels(train_notepad_path)

# Combine test and train labels
all_session_labels = {**test_session_labels, **train_session_labels}'''

"# Function to read session IDs and their labels\ndef read_session_labels(file_path):\n    session_labels = {}\n    with open(file_path, 'r') as file:\n        for line in file:\n            parts = line.strip().split()\n            if len(parts) == 2:\n                session_id, label = parts\n                session_labels[session_id] = label\n    return session_labels\n\n# Read session IDs and labels from both test and train files\ntest_session_labels = read_session_labels(test_notepad_path)\ntrain_session_labels = read_session_labels(train_notepad_path)\n\n# Combine test and train labels\nall_session_labels = {**test_session_labels, **train_session_labels}"

In [153]:
'''import csv

# Define input and output file paths
input_file = 'phase2/annotations/humans_and_moderate_and_advanced_bots/humans_and_moderate_and_advanced_bots.txt'  # Path to your text file
output_file = 'phase2/annotations/humans_and_moderate_and_advanced_bots/humans_and_moderate_and_advanced_bots.csv'  # Path for the output CSV file

# Read data from the input text file
with open(input_file, 'r') as file:
    lines = file.readlines()

# Process the data into a list of tuples (session_id, result)
data = [tuple(line.strip().split()) for line in lines]

# Write the processed data into the CSV file
with open(output_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['session_id', 'result'])  # Write the header
    writer.writerows(data)  # Write the session data

print(f'Data has been converted to {output_file}')'''


"import csv\n\n# Define input and output file paths\ninput_file = 'phase2/annotations/humans_and_moderate_and_advanced_bots/humans_and_moderate_and_advanced_bots.txt'  # Path to your text file\noutput_file = 'phase2/annotations/humans_and_moderate_and_advanced_bots/humans_and_moderate_and_advanced_bots.csv'  # Path for the output CSV file\n\n# Read data from the input text file\nwith open(input_file, 'r') as file:\n    lines = file.readlines()\n\n# Process the data into a list of tuples (session_id, result)\ndata = [tuple(line.strip().split()) for line in lines]\n\n# Write the processed data into the CSV file\nwith open(output_file, mode='w', newline='') as file:\n    writer = csv.writer(file)\n    writer.writerow(['session_id', 'result'])  # Write the header\n    writer.writerows(data)  # Write the session data\n\nprint(f'Data has been converted to {output_file}')"

In [154]:
# Define the paths to the CSV files
test_csv_paths = ['phase1/annotations/humans_and_advanced_bots/test.csv',
                 'phase1/annotations/humans_and_moderate_bots/test.csv',
                 #'G:\web_bot_detection_dataset\phase2\\annotations\humans_and_advanced_bots\\humans_and_advanced_bots.csv'
                 ]

train_csv_paths = ['phase1/annotations/humans_and_advanced_bots/train.csv',
                  'phase1/annotations/humans_and_moderate_bots/train.csv',
                  #'G:\web_bot_detection_dataset\phase2\\annotations\humans_and_moderate_and_advanced_bots\\humans_and_moderate_and_advanced_bots.csv'
                  ]

json_directory_path = ['phase1/data/mouse_movements/humans_and_advanced_bots',
                       'phase1/data/mouse_movements/humans_and_moderate_bots',
                       #'G:\web_bot_detection_dataset\phase2\data\mouse_movements\\bots',
                       #'G:\web_bot_detection_dataset\phase2\data\mouse_movements\\bots'
                       ]

# Function to load session data from multiple CSV files
def load_session_data(csv_paths):
    data_frames = []
    for path in csv_paths:
        df = pd.read_csv(path)
        data_frames.append(df[['session_id', 'result']])  # Load only the session_id and result columns
    return pd.concat(data_frames, ignore_index=True)  # Combine all dataframes into one

# Load train and test datasets
train_df = load_session_data(train_csv_paths)
test_df = load_session_data(test_csv_paths)

# Check the combined dataframes
print(train_df.head())
print(test_df.head())

                   session_id result
0  dr09rk5eagjuu87gedvdqmq3gl  human
1  gq715ms79515gcq39vf91mli6t  human
2  hrbko2t4t14q3pahqltndlolb5  human
3  nvmlnfhs5v6hehsd81e9mf75cn  human
4  brrlh9tmiodt2ekkjvn7kcsps0  human
                   session_id result
0  g2gh9qmk9krld14h5uojlg7g10  human
1  kaodsjbnqm7umgfvao63d3rihb  human
2  1aqgqrcuurlmvvbbpirvsh7e53  human
3  igbeqcjnbst8afmoi4sg6tn669  human
4  vopb1c4o3o2dpsov8jinbbou5h  human


In [155]:
def load_mouse_movement_data(session_id, base_paths):
    for base_path in base_paths:
        file_path = os.path.join(base_path, session_id, 'mouse_movements.json')
        if os.path.isfile(file_path):
            try:
                with open(file_path, 'r') as f:
                    mouse_data = json.load(f)
                return mouse_data
            except json.JSONDecodeError:
                print(f"Error decoding JSON from file: {file_path}")
                continue
    print(f"File not found for session ID {session_id} in any of the provided paths.")
    return None

In [156]:
import numpy as np
import re

import re

def parse_mouse_movements(total_behaviour):
    # Regular expression to extract coordinates
    pattern = re.compile(r'\[m\((\d+),(\d+)\)\]')
    matches = pattern.findall(total_behaviour)
    
    # Convert to list of tuples
    coordinates = [(int(x), int(y)) for x, y in matches]
    
    return coordinates

def extract_lstm_features(mouse_data):
    # Parse the mouse movements
    coordinates = parse_mouse_movements(mouse_data.get('total_behaviour', ''))
    
    if not coordinates:
        return np.zeros((20, 2))  # Return a zero-filled array if no movements

    x_coords, y_coords = zip(*coordinates) if coordinates else ([], [])

    # Normalize or pad the sequences to a fixed length for LSTM
    max_length = 20  # Example length
    x_coords = list(x_coords[:max_length]) + [0] * (max_length - len(x_coords))
    y_coords = list(y_coords[:max_length]) + [0] * (max_length - len(y_coords))

    # Combine x and y coordinates into a 2D array
    features = np.column_stack((x_coords, y_coords))

    return features


In [157]:
def extract_features_from_multiple_paths(df, base_paths):
    features, labels = [], []
    for session_id, label in zip(df['session_id'], df['result']):
        mouse_data = load_mouse_movement_data(session_id, base_paths)
        if mouse_data:
            features.append(extract_lstm_features(mouse_data))
            labels.append(1 if label == 'human' else 0)
    return np.array(features), np.array(labels)

def load_session_data(csv_paths):
    data_frames = []
    for path in csv_paths:
        df = pd.read_csv(path)
        data_frames.append(df[['session_id', 'result']])
    return pd.concat(data_frames, ignore_index=True)


train_df = load_session_data(train_csv_paths)
test_df = load_session_data(test_csv_paths)

train_features, train_labels = extract_features_from_multiple_paths(train_df, json_directory_path)
test_features, test_labels = extract_features_from_multiple_paths(test_df, json_directory_path)

print(f"Train features shape: {train_features.shape}")
print(f"Test features shape: {test_features.shape}")

Train features shape: (140, 20, 2)
Test features shape: (60, 20, 2)


In [158]:
def prepare_data(csv_paths, base_paths):
    all_features = []
    all_labels = []

    for csv_path in csv_paths:
        df = load_session_data([csv_path])  # Pass the single path as a list

        for _, row in df.iterrows():
            session_id = row['session_id']
            label = row['result']

            mouse_data = load_mouse_movement_data(session_id, base_paths)
            if mouse_data:
                features = extract_lstm_features(mouse_data)
                all_features.append(features)
                all_labels.append(1 if label == 'human' else 0)  # 1 for human, 0 for bot

    return np.array(all_features), np.array(all_labels)


# Use file paths directly, not DataFrames
train_features, train_labels = prepare_data(train_csv_paths, json_directory_path)
test_features, test_labels = prepare_data(test_csv_paths, json_directory_path)
print(train_df['result'].value_counts())


result
human           70
advanced_bot    35
moderate_bot    35
Name: count, dtype: int64


In [159]:
from keras.utils import to_categorical

# One-hot encode the labels
train_labels = to_categorical(train_labels, num_classes=2)
test_labels = to_categorical(test_labels, num_classes=2)

# Build the LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(20, 2), return_sequences=True))
model.add(LSTM(64))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_features, train_labels, epochs=10, batch_size=32, validation_data=(test_features, test_labels))


Epoch 1/10


c:\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 79ms/step - accuracy: 0.6311 - loss: 0.6241 - val_accuracy: 0.9833 - val_loss: 0.4239
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 1.0000 - loss: 0.3867 - val_accuracy: 1.0000 - val_loss: 0.2440
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 1.0000 - loss: 0.2168 - val_accuracy: 1.0000 - val_loss: 0.1015
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 1.0000 - loss: 0.0909 - val_accuracy: 1.0000 - val_loss: 0.0399
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 1.0000 - loss: 0.0356 - val_accuracy: 1.0000 - val_loss: 0.0164
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 1.0000 - loss: 0.0160 - val_accuracy: 1.0000 - val_loss: 0.0071
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 1.0000 - loss: 0.0066 - val_accuracy: 1.0000 - val_loss: 0.0036
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 1.0000 - loss: 0.0035 - val_accuracy: 1.0000 - val_loss: 0.0022
Epoch 9/10


In [160]:
# Predict the test data
test_predictions = model.predict(test_features)
test_predictions = np.argmax(test_predictions, axis=1)
true_labels = np.argmax(test_labels, axis=1)

# Calculate accuracy and print classification report
accuracy = accuracy_score(true_labels, test_predictions)
print(f"Accuracy: {accuracy}")
print("Classification Report:\n", classification_report(true_labels, test_predictions, target_names=['bot', 'human']))


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

         bot       1.00      1.00      1.00        30
       human       1.00      1.00      1.00        30

    accuracy                           1.00        60
   macro avg       1.00      1.00      1.00        60
weighted avg       1.00      1.00      1.00        60



In [161]:
# Save the trained model
model.save('lstm_mouse_movement_classifier.h5')


In [162]:
import re
import numpy as np
from keras.models import load_model

# Sample test session
test_session = {
    "session_id": "jfmilo33fin84baeh3k6bcnh3v",
    "total_behaviour": "[m(0,4)][m(0,5)][m(0,6)][m(0,7)][m(1,8)][m(1,9)][m(1,10)]"
}

# Function to load and extract features for a single session
def load_mouse_movement_data_from_sample(test_session):
    return test_session['total_behaviour']

# Feature extraction remains the same
def extract_lstm_features(mouse_movements):
    x_coords, y_coords = [], []

    # Use regular expressions to find all coordinate entries
    matches = re.findall(r'm\((\d+),(\d+)\)', mouse_movements)

    if not matches:
        return np.zeros((20, 2))  # Return a zero-filled array if no movements

    # Convert matches to integer lists
    for match in matches:
        x, y = map(int, match)
        x_coords.append(x)
        y_coords.append(y)

    # Normalize or pad the sequences to a fixed length for LSTM
    max_length = 20  # Example length
    x_coords = x_coords[:max_length] + [0] * (max_length - len(x_coords))
    y_coords = y_coords[:max_length] + [0] * (max_length - len(y_coords))

    # Combine x and y coordinates into a 2D array
    features = np.column_stack((x_coords, y_coords))

    return features

# Extract features for the test session
test_mouse_data = load_mouse_movement_data_from_sample(test_session)
test_features = extract_lstm_features(test_mouse_data)

# Reshape the features to 3D (1, 20, 2) for a single test sample
test_features = np.expand_dims(test_features, axis=0)

# Load the trained model
model = load_model('lstm_mouse_movement_classifier.h5')

# Make predictions
test_prediction = model.predict(test_features)
predicted_label = np.argmax(test_prediction, axis=1)[0]

# Map the label back to human or bot
label_mapping = {0: 'bot', 1: 'human'}
print(f"Predicted Label: {label_mapping[predicted_label]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
Predicted Label: bot


In [163]:
import re
import numpy as np
from keras.models import load_model

# Sample test session
test_session = {
    "session_id": "g2gh9qmk9krld14h5uojlg7g10",
    "total_behaviour": "[m(627,558)][m(626,555)][m(625,552)][m(624,548)][m(622,539)][m(622,535)][m(622,531)][m(622,525)][m(620,518)][m(614,506)][m(607,496)][m(602,488)][m(601,483)][m(601,482)][m(601,481)][m(600,479)][m(600,476)][m(598,475)][m(595,472)][m(592,469)][m(587,467)][m(583,467)][m(578,467)][m(576,466)][m(574,465)][m(572,465)][m(567,464)][m(558,461)][m(551,458)][m(547,455)][m(545,453)][m(542,448)][m(538,441)][m(533,435)]"
}

# Function to load and extract features for a single session
def load_mouse_movement_data_from_sample(test_session):
    return test_session['total_behaviour']

# Feature extraction remains the same
def extract_lstm_features(mouse_movements):
    x_coords, y_coords = [], []

    # Use regular expressions to find all coordinate entries
    matches = re.findall(r'm\((\d+),(\d+)\)', mouse_movements)

    if not matches:
        return np.zeros((20, 2))  # Return a zero-filled array if no movements

    # Convert matches to integer lists
    for match in matches:
        x, y = map(int, match)
        x_coords.append(x)
        y_coords.append(y)

    # Normalize or pad the sequences to a fixed length for LSTM
    max_length = 20  # Example length
    x_coords = x_coords[:max_length] + [0] * (max_length - len(x_coords))
    y_coords = y_coords[:max_length] + [0] * (max_length - len(y_coords))

    # Combine x and y coordinates into a 2D array
    features = np.column_stack((x_coords, y_coords))

    return features

# Extract features for the test session
test_mouse_data = load_mouse_movement_data_from_sample(test_session)
test_features = extract_lstm_features(test_mouse_data)

# Reshape the features to 3D (1, 20, 2) for a single test sample
test_features = np.expand_dims(test_features, axis=0)

# Load the trained model
model = load_model('lstm_mouse_movement_classifier.h5')

# Make predictions
test_prediction = model.predict(test_features)
predicted_label = np.argmax(test_prediction, axis=1)[0]

# Map the label back to human or bot
label_mapping = {0: 'bot', 1: 'human'}
print(f"Predicted Label: {label_mapping[predicted_label]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
Predicted Label: human
